# ProRes Frames to Azure Blob Example
Here we write original ProRes compressed frames into a flat Azure blob store and show how to interact with these data using lazy operations.

#### Build a list of frames to transfer using the dbcamhd.json database

In [ ]:
import numpy as np
import pandas as pd
import pycamhd as camhd
dbcamhd = pd.read_json('dbcamhd.json', orient='records', lines=True)
dbcamhd.tail()

In [ ]:
fileindex = 2064
filename = dbcamhd.filename[fileindex]
timestamp = dbcamhd.timestamp[fileindex]
frame_count = dbcamhd.frame_count[fileindex]
n_images = 1000
frame_numbers = np.linspace(750,frame_count-6000, n_images, dtype=np.int64())
frame_numbers = np.arange(20000, frame_count)

#### Create blob service

In [ ]:
account_name = 'camhd'
container_name = 'prores'
with open('.camhd_account_key', 'r') as f:
    account_key = f.read()
from azure.storage.blob import BlockBlobService
blob_service = BlockBlobService(account_name, account_key)

#### ProRes to blob delayed functions

In [ ]:
from dask import delayed, compute

@delayed
def delayed_get_frame_data(filename, frame_number, moov_atom):
    return camhd.get_frame_data(filename, frame_number, moov_atom)

@delayed
def delayed_prores_to_blob(blob_service, container_name, blob_name, frame_data):
    blob_service.create_blob_from_bytes(container_name, blob_name, frame_data)
    return 0

#### Set list of delayed functions

In [ ]:
moov_atom = camhd.get_moov_atom(filename)
delayed_transfer = []
for frame_number in frame_numbers:
    frame_data = delayed_get_frame_data(filename, frame_number, moov_atom)
    blob_name = '%i-%08.0f' % (timestamp, frame_number)
    delayed_transfer.append(delayed_prores_to_blob(blob_service, container_name, blob_name, frame_data))

In [ ]:
delayed_transfer[0]

#### Start a Dask cluster

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=70)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

#### Transfer frames

In [ ]:
%%time
codes = compute(*delayed_transfer)

#### Count blobs in container

In [ ]:
len(list(blob_service.list_blobs(container_name)))

In [ ]:
frame_count

#### Get list of frames not in container

In [ ]:
frames_transferred = []
for blob in blob_service.list_blobs(container_name):
    frames_transferred.append(int(blob.name.split('-')[1]))

In [ ]:
frames_needed = []
for frame_number in range(frame_count):
    if frame_number not in frames_transferred:
        frames_needed.append(frame_number)

In [ ]:
frame_numbers = frames_needed

#### Delete all blobs in container

In [ ]:
#for blob in blob_service.list_blobs(container_name):
#    blob_service.delete_blob(container_name, blob.name)